In [0]:
# Classificação de Sentimentos - Tensor Flow

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
!pip install category_encoders


     |████████████████████████████████| 102kB 2.3MB/s 


In [3]:
!tree


/bin/bash: tree: command not found


In [0]:

df_train = pd.read_csv("https://raw.githubusercontent.com/nilberthsouza/Classificador-de-Sentimentos/master/Datasets/trainingdatasets/Train500.csv",encoding='utf-8', sep=';',engine='python')
df_train.head()
stopword = pd.read_table("https://raw.githubusercontent.com/nilberthsouza/Classificador-de-Sentimentos/master/stopwords.txt")

In [46]:
df_train.tail()

,id,tweet_text,tweet_date,sentiment,query_used
499995,1039281130304880640,@joaomedina hahahahahha capaz viu :( ele é chato,Mon Sep 10 22:35:05 +0000 2018,0,:(
499996,1039281126987231232,eu so queria as fotos que tirei nesse feriado :(,Mon Sep 10 22:35:04 +0000 2018,0,:(
499997,1039281117017325568,@yasmimleal19 viajou hoje :(,Mon Sep 10 22:35:01 +0000 2018,0,:(
499998,1039281105977917440,triste saber que o jimin e os meninos tem bast...,Mon Sep 10 22:34:59 +0000 2018,0,:(
499999,1039281104782536705,"Vejo muito casal gay aqui, choque. Só eu que c...",Mon Sep 10 22:34:58 +0000 2018,0,:(


In [47]:
df_train.drop(['id','tweet_date','query_used'],axis=1,inplace=True)
df_train.head(5)

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1


In [48]:
df_train = df_train.sample(frac=1)
df_train = df_train.sample(frac=1)
df_train = df_train.sample(frac=1)
df_train = df_train.sample(frac=1)
df_train = df_train.reset_index()

df_train.head(80)

,index,tweet_text,sentiment
0,15621,@RaulTorresPLD Felicidades :),1
1,164917,"@canibalindia substitua ""mandar um meme"" por ""...",1
2,463819,To chorando com sdd da minh namorada manoo :(,0
3,280399,olha esses olhinhos que coisa mais fofa :(( ht...,0
4,444471,só de pensar que amanhã eu fico até as 6hrs da...,0
...,...,...,...
75,445092,os amgs que gosto estao bebendo sem mim :(,0
76,50884,@ltcm_ eu uso esses dois mesmo mas achei o des...,1
77,66419,@rubina_eb Este desGoverno dá bons mêmes :),1
78,352173,@ognajoao_ Kkkkkkkk :( vdd Vc vai né?,0


In [0]:
stoplist = []
sentences = []
for i in range(len(stopword)):
    stoplist.append(stopword['stopwords'][i])
    
for i in range(len(df_train['tweet_text'])):
    sentence = df_train['tweet_text'][i]
    for word in stoplist:
        newword = " "+word+" "
        sentence = sentence.replace(newword," ")
        sentence = sentence.replace(newword.upper(),'')
        sentence = sentence.replace("  "," ")
    sentences.append(sentence)
    
 

agora vamos remover os emojis

In [0]:
import re
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


In [51]:
sentences = list(map(emoji,sentences))
sentences[0]

'@RaulTorresPLD Felicidades :)'

In [0]:
training_sentences = []
training_labels = []

training_sentences = sentences

#sentimentos negativos estão com 2 no dataset, vamos definir como 0



training_labels = df_train['sentiment']

In [53]:

#training_labels[8]
df_train.head(20)

,index,tweet_text,sentiment
0,15621,@RaulTorresPLD Felicidades :),1
1,164917,"@canibalindia substitua ""mandar um meme"" por ""...",1
2,463819,To chorando com sdd da minh namorada manoo :(,0
3,280399,olha esses olhinhos que coisa mais fofa :(( ht...,0
4,444471,só de pensar que amanhã eu fico até as 6hrs da...,0
5,216890,@gomesiris07 Tenta agora :),1
6,115284,"Oi pessoas, se vocês quiserem comprar inlumina...",1
7,108031,No q vc se inspira pra se vestir ?! Montar loo...,1
8,172443,literalmente vendo nesss app lugares baratos p...,1
9,113138,foo :-) : 1872aa1a-ae95-4243-893c-a24a1f2de0ae,1


In [54]:
training_labels = np.array(training_labels)
training_labels

array([1, 1, 0, ..., 0, 0, 1])

In [55]:
x_training, x_validation, y_training, y_validation = train_test_split(training_sentences,training_labels,test_size=0.3,random_state=17)
print(len(x_training),len(y_training), len(x_validation), len(y_validation))

350000 350000 150000 150000


In [0]:

vocab_size = 10000
embedding_dim = 16
max_length = 240
trunc_type = 'post'
oov_tok = "<UNK>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok,)
tokenizer.fit_on_texts(x_training)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_training)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(x_validation)
validation_padded = pad_sequences(testing_sequences,maxlen=max_length,truncating=trunc_type)

In [57]:
padded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [63]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 240, 16)           160000    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               41472     
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 774       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 7         
Total params: 202,253
Trainable params: 202,253
Non-trainable params: 0
_________________________________________________________________


In [64]:

num_epochs = 5
history = model.fit(padded, y_training,epochs=num_epochs,validation_data=(validation_padded,y_validation))

Epoch 1/5
 6879/10938 [=================>............] - ETA: 1:24 - loss: 0.4765 - accuracy: 0.7615

KeyboardInterrupt: ignored

In [62]:
sentencia = ["odio"]
sequencia = tokenizer.texts_to_sequences(sentencia)
pren = pad_sequences(sequencia, maxlen=max_length,padding=trunc_type)
print(model.predict(pren))

[[0.49995375]]
